Analyzing Jeopardy Questions trying to find out whether I can get an edge when playing!
Data being used: https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file/

In [646]:

import pandas as pd
# Add asset from file system
jeopardy = pd.read_csv('../datasets/JEOPARDY_CSV.csv')
jeopardy = jeopardy.iloc[0:65000]
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [647]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [648]:
jeopardy.columns = jeopardy.columns.str.replace(" ","")

In [649]:
jeopardy.columns

Index(['ShowNumber', 'AirDate', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')

In [650]:
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Data columns (total 7 columns):
ShowNumber    65000 non-null int64
AirDate       65000 non-null object
Round         65000 non-null object
Category      65000 non-null object
Value         65000 non-null object
Question      65000 non-null object
Answer        65000 non-null object
dtypes: int64(1), object(6)
memory usage: 3.5+ MB


In [651]:
jeopardy["Question"].head()

0    For the last 8 years of his life, Galileo was ...
1    No. 2: 1912 Olympian; football star at Carlisl...
2    The city of Yuma in this state has a record av...
3    In 1963, live on "The Art Linkletter Show", th...
4    Signer of the Dec. of Indep., framer of the Co...
Name: Question, dtype: object

In [652]:
jeopardy["Answer"].head()

0    Copernicus
1    Jim Thorpe
2       Arizona
3    McDonald's
4    John Adams
Name: Answer, dtype: object

In [653]:
def str_normalizer(data): #to lowercase and clean the string
    data = str(data)
    lowered = data.lower()
    return lowered

jeopardy["clean_question"] = jeopardy["Question"].apply(str_normalizer)

In [654]:
jeopardy["clean_question"].head()

0    for the last 8 years of his life, galileo was ...
1    no. 2: 1912 olympian; football star at carlisl...
2    the city of yuma in this state has a record av...
3    in 1963, live on "the art linkletter show", th...
4    signer of the dec. of indep., framer of the co...
Name: clean_question, dtype: object

In [655]:
jeopardy["clean_answer"] = jeopardy["Answer"].apply(str_normalizer)

In [656]:
jeopardy["clean_answer"].head()

0    copernicus
1    jim thorpe
2       arizona
3    mcdonald's
4    john adams
Name: clean_answer, dtype: object

In [657]:
jeopardy["Value"].unique()

array(['$200', '$400', '$600', '$800', '$2,000', '$1000', '$1200',
       '$1600', '$2000', '$3,200', 'None', '$5,000', '$100', '$300',
       '$500', '$1,000', '$1,500', '$1,200', '$4,800', '$1,800', '$1,100',
       '$2,200', '$3,400', '$3,000', '$4,000', '$1,600', '$6,800',
       '$1,900', '$3,100', '$700', '$1,400', '$2,800', '$8,000', '$6,000',
       '$2,400', '$12,000', '$3,800', '$2,500', '$6,200', '$10,000',
       '$7,000', '$1,492', '$7,400', '$1,300', '$7,200', '$2,600',
       '$3,300', '$5,400', '$4,500', '$2,100', '$900', '$3,600', '$2,127',
       '$367', '$4,400', '$3,500', '$2,900', '$3,900', '$4,100', '$4,600',
       '$10,800', '$2,300', '$5,600', '$1,111', '$8,200', '$5,800',
       '$750', '$7,500', '$1,700', '$9,000', '$6,100', '$1,020', '$4,700',
       '$2,021', '$5,200', '$3,389', '$4,200', '$5', '$2,001', '$1,263',
       '$4,637', '$3,201', '$6,600', '$3,700', '$2,990', '$5,500',
       '$14,000', '$2,700', '$6,400', '$350', '$8,600', '$6,300', '$250',
    

In [658]:
def num_normalizer(data): #to make column numeric
    try:
        data = str(data)
        data = data.replace("$","")
        data = data.replace(",","")
        return int(data)
    except Exception:
        return 0

In [659]:
jeopardy["clean_value"] = jeopardy["Value"].apply(num_normalizer)

In [660]:
jeopardy["clean_value"].head()

0    200
1    200
2    200
3    200
4    200
Name: clean_value, dtype: int64

In [661]:
jeopardy["AirDate"].head()

0    2004-12-31
1    2004-12-31
2    2004-12-31
3    2004-12-31
4    2004-12-31
Name: AirDate, dtype: object

In [662]:
jeopardy["air_date_dt"] = pd.to_datetime(jeopardy["AirDate"],infer_datetime_format=True)

In [663]:
jeopardy["air_date_dt"].head()

0   2004-12-31
1   2004-12-31
2   2004-12-31
3   2004-12-31
4   2004-12-31
Name: air_date_dt, dtype: datetime64[ns]

In [664]:
def word_count(series):
    split_answer = series["clean_answer"].split(" ")
    split_question = series["clean_question"].split(" ")
    match_count = 0
    if "the" in split_answer:
        split_answer.remove("the")
    for word_a in split_answer:
        if word_a in split_question:
            match_count += 1
    if len(split_answer) > 0:
        matcher = match_count / len(split_answer)
    else: 
        matcher = 0
    return matcher

jeopardy["answer_in_question"] = jeopardy.apply(word_count, axis=1)

In [665]:
jeopardy["answer_in_question"].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: answer_in_question, dtype: float64

In [666]:
jeopardy["answer_in_question"].describe()

count    65000.000000
mean         0.045555
std          0.142304
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: answer_in_question, dtype: float64

In [667]:
jeopardy.sort_values(by="answer_in_question", ascending=False).head(10)

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value,air_date_dt,answer_in_question
13432,4726,2005-03-07,Jeopardy!,WHO'S HAD MORE NO. 1 HITS?,$400,"George Michael, Phil Collins, Elton John",Elton John,"george michael, phil collins, elton john",elton john,400,2005-03-07,1.0
43931,3830,2001-04-06,Jeopardy!,SO YOU WANT TO JOIN THE LAPD,$300,"Of weight, height or hearing requirements, the...",Height,"of weight, height or hearing requirements, the...",height,300,2001-04-06,1.0
48738,5548,2008-10-22,Jeopardy!,PUT 'EM IN ORDER,$800,"By founding: London, Paris, New York City","Paris, London, New York City","by founding: london, paris, new york city","paris, london, new york city",800,2008-10-22,1.0
6248,5307,2007-10-09,Jeopardy!,NOT A VERB,$600,"Wind, wander, wonderful",wonderful,"wind, wander, wonderful",wonderful,600,2007-10-09,1.0
6254,5307,2007-10-09,Jeopardy!,NOT A VERB,$800,"Salt, pepper, sage",sage,"salt, pepper, sage",sage,800,2007-10-09,1.0
48720,5548,2008-10-22,Jeopardy!,PUT 'EM IN ORDER,$200,"Statehood: Florida, Virginia, Nebraska","Virginia, Florida, Nebraska","statehood: florida, virginia, nebraska","virginia, florida, nebraska",200,2008-10-22,1.0
4797,5686,2009-05-04,Double Jeopardy!,TIME TO CONVERT,"$2,200",You do the math: -40 degrees on the Fahrenheit...,-40,you do the math: -40 degrees on the fahrenheit...,-40,2200,2009-05-04,1.0
56504,5696,2009-05-18,Jeopardy!,STUPID ANSWERS: THE BRITISH EDITION,$200,A reservoir called the Queen's Basin once grac...,Green Park,a reservoir called the queen's basin once grac...,green park,200,2009-05-18,1.0
56510,5696,2009-05-18,Jeopardy!,STUPID ANSWERS: THE BRITISH EDITION,$400,The Garden Tower at the Tower of London was th...,the Bloody Tower,the garden tower at the tower of london was th...,the bloody tower,400,2009-05-18,1.0
56516,5696,2009-05-18,Jeopardy!,STUPID ANSWERS: THE BRITISH EDITION,$600,"Battle Abbey, built on the spot where King Har...",Battle,"battle abbey, built on the spot where king har...",battle,600,2009-05-18,1.0


In [668]:
jeopardy.groupby(by="Category").mean()["answer_in_question"].sort_values(ascending=False)

Category
STUPID GEOGRAPHIC ANSWERS              1.000000
ACCOUNTING EXAM                        1.000000
STUPID ANSWERS: THE BRITISH EDITION    1.000000
STUPID ANSWERS AT THE MOVIES           0.900000
NOT A POPE                             0.875000
WHICH CAME FIRST?                      0.816667
WHO'S HAD MORE NO. 1 HITS?             0.700000
MIDDLE EASTERN AFFAIRS                 0.666667
PUT 'EM IN ORDER                       0.651243
OF ORDER                               0.634286
NOT A CURRENT NATIONAL CAPITAL         0.600000
SEXPERTISE                             0.600000
NOT A VERB                             0.600000
BIRD HUNTING                           0.600000
THE SOUTHERNMOST CAPITAL CITY          0.550000
THE SMALLEST IN AREA                   0.547619
BORN FIRST                             0.516667
FIX THE PROVERB                        0.514048
LET'S HAVE ORDER                       0.506667
STUPID ANSWERS                         0.506667
MEDICAL ISSUES                 

Approximatly 0.1 words from the question occur in the answer. This is not very enlightening and helpful. It's worth looking at the answers where the most words of the question reappear. It shows that there is a category "PUT 'EM IN ORDER" which drives up the average significantly. Moreover the category "FIX THE PROVERB" and "PROVERBS" repeat a lot of words which, again, lies in the nature of the category. So question 1 "How often the answer is deducible from the question." can be answered with it depends on the category of the question. 

In [669]:
jeopardy_date_sorted = jeopardy.sort_values(by="air_date_dt")

In [670]:
jeopardy_date_sorted.head()

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value,air_date_dt,answer_in_question
19281,10,1984-09-21,Jeopardy!,DOUBLE TALK,$200,Affirmative reply to an admiral's command,aye-aye,affirmative reply to an admiral's command,aye-aye,200,1984-09-21,0.0
19297,10,1984-09-21,Jeopardy!,"""JACKS"" OF ALL TRADES",$500,"2nd generation pop singer, he's sung on every ...",Jack Jones,"2nd generation pop singer, he's sung on every ...",jack jones,500,1984-09-21,0.0
19296,10,1984-09-21,Jeopardy!,DOUBLE TALK,$500,British rockers whose name came from villain i...,Duran Duran,british rockers whose name came from villain i...,duran duran,500,1984-09-21,0.0
19295,10,1984-09-21,Jeopardy!,GEOGRAPHY,$500,About 1/3 of Africa's Pygmy population lives i...,Zaire,about 1/3 of africa's pygmy population lives i...,zaire,500,1984-09-21,0.0
19294,10,1984-09-21,Jeopardy!,"""B"" MOVIES",$400,This title describes Carroll Baker's thumb-suc...,Baby Doll,this title describes carroll baker's thumb-suc...,baby doll,400,1984-09-21,0.0


In [671]:
terms_used = set()
question_overlap = []

In [672]:
question_overlap = []
terms_used = []
for i, row in jeopardy_date_sorted.iterrows():
  split_question = row['clean_question'].split(' ')
  split_question = [q for q in split_question if len(q) > 5]
  match_count = 0
  for q in split_question:
    if q in terms_used:
      match_count += 1
    else: terms_used.append(q)
  if len(split_question) > 0:
    question_overlap.append(match_count / len(split_question))
  else:
    question_overlap.append(0)
jeopardy_date_sorted['question_overlap'] = question_overlap
jeopardy_date_sorted.sort_values(by='question_overlap', ascending=False).head()

,ShowNumber,AirDate,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value,air_date_dt,answer_in_question,question_overlap
12514,4586,2004-07-12,Jeopardy!,THE 1860s,$1000,"On March 17, 1861 this last king of Sardinia b...",Victor Emmanuel,"on march 17, 1861 this last king of sardinia b...",victor emmanuel,1000,2004-07-12,0.0,1.0
21871,4929,2006-02-02,Double Jeopardy!,2-SPORT TERMS,$800,"In 2 different sports, it can weigh 1 1/8 ounc...",a shot,"in 2 different sports, it can weigh 1 1/8 ounc...",a shot,800,2006-02-02,0.0,1.0
21846,4929,2006-02-02,Jeopardy!,"""F"" STOP",$400,An amusement park ride consisting of an uprigh...,a Ferris wheel,an amusement park ride consisting of an uprigh...,a ferris wheel,400,2006-02-02,0.0,1.0
21845,4929,2006-02-02,Jeopardy!,I AM A CAMERA,$400,"This ""colorful"" camera introduced in 1900 cost...",the Brownie,"this ""colorful"" camera introduced in 1900 cost...",the brownie,400,2006-02-02,0.0,1.0
21844,4929,2006-02-02,Jeopardy!,FIRST LADIES,$400,"Her father was John V. Bouvier III, a New York...",Jackie Kennedy,"her father was john v. bouvier iii, a new york...",jackie kennedy,400,2006-02-02,0.0,1.0


In [673]:
def value_count(row):
    if row["clean_value"] > 800:
        value = 1
    else:
        value = 0
    return value

jeopardy["high_value"] = jeopardy.apply(value_count, axis=1)
        

In [674]:
def word_eva(word, df):
    low_count = 0
    high_count = 0
    for i, row in df.iterrows():
        splitted_question = row["clean_question"].split(" ")
        if word in splitted_question:
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return (high_count, low_count)

observed_expected = []
observed_expected_dict = {}
comparison_terms = list(terms_used[:8])
for item in comparison_terms:
    temp_list = word_eva(item, jeopardy)
    observed_expected.append(temp_list)
    observed_expected_dict[item] = temp_list

In [675]:
observed_expected_dict

{"admiral's": (0, 2),
 'affirmative': (0, 2),
 'british': (190, 395),
 'command': (14, 22),
 'generation': (2, 10),
 'rockers': (3, 3),
 'singer,': (4, 8),
 'voyage': (2, 27)}

In [676]:
observed_expected

[(0, 2), (0, 2), (14, 22), (2, 10), (4, 8), (2, 27), (190, 395), (3, 3)]

In [677]:
low_value_count = jeopardy["high_value"].value_counts()[0]
high_value_count = jeopardy["high_value"].value_counts()[1]
jeopardy["high_value"].value_counts()

0    46416
1    18584
Name: high_value, dtype: int64

In [678]:
#new = (1,100) # testing whether chi works as expected
#observed_expected.append(new)
#observed_expected

In [679]:
chi_squared = []
from scipy.stats import chisquare

In [681]:
for item in observed_expected:
    total = sum(item)
    total_prop = total / len(jeopardy)
    expected_count_high = total_prop * high_value_count
    expected_count_low = total_prop * low_value_count
    chi_square_values = chisquare(item, f_exp=[expected_count_high, expected_count_low])
    chi_squared.append(chi_square_values)

chi_squared

[Power_divergenceResult(statistic=0.8007583591864874, pvalue=0.37086672928489395),
 Power_divergenceResult(statistic=0.8007583591864874, pvalue=0.37086672928489395),
 Power_divergenceResult(statistic=1.8699852172768794, pvalue=0.17147677260184616),
 Power_divergenceResult(statistic=0.8357039539213132, pvalue=0.36062833980107545),
 Power_divergenceResult(statistic=0.1321987870744269, pvalue=0.7161630665709534),
 Power_divergenceResult(statistic=6.685067290041586, pvalue=0.009722385287789946),
 Power_divergenceResult(statistic=4.3310942270283235, pvalue=0.03742218114782494),
 Power_divergenceResult(statistic=1.3470173272891335, pvalue=0.24580022922966832),
 Power_divergenceResult(statistic=0.8007583591864874, pvalue=0.37086672928489395),
 Power_divergenceResult(statistic=0.8007583591864874, pvalue=0.37086672928489395),
 Power_divergenceResult(statistic=1.8699852172768794, pvalue=0.17147677260184616),
 Power_divergenceResult(statistic=0.8357039539213132, pvalue=0.36062833980107545),
 Powe

In [682]:
git

NameError: name 'git' is not defined